In [196]:
import pandas as pd

df = pd.read_csv('../data/processed/feature_eng_weather_data.csv')
print("Processed dataset loaded successfully!")


Processed dataset loaded successfully!


In [123]:
df.head()

,country,location_name,latitude,longitude,timezone,last_updated,temperature_celsius,wind_kph,wind_degree,wind_direction,pressure_mb,pressure_in,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination,condition_text_Blowing snow,condition_text_Clear,condition_text_Cloudy,condition_text_Fog,condition_text_Freezing drizzle,condition_text_Freezing fog,condition_text_Heavy rain,condition_text_Heavy rain at times,...,condition_text_Moderate snow,condition_text_Overcast,condition_text_Partly Cloudy,condition_text_Partly cloudy,condition_text_Patchy heavy snow,condition_text_Patchy light drizzle,condition_text_Patchy light rain,condition_text_Patchy light rain in area with thunder,condition_text_Patchy light rain with thunder,condition_text_Patchy light snow,condition_text_Patchy light snow in area with thunder,condition_text_Patchy moderate snow,condition_text_Patchy rain nearby,condition_text_Patchy rain possible,condition_text_Patchy snow nearby,condition_text_Patchy snow possible,condition_text_Sunny,condition_text_Thundery outbreaks in nearby,condition_text_Thundery outbreaks possible,condition_text_Torrential rain shower,hour,day,month,year,temp_diff,wind_power,air_quality_risk,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,temperature_celsius_lag_1,temperature_celsius_lag_7,humidity_lag_1,humidity_lag_7,precip_mm_lag_1,precip_mm_lag_7,distance_from_equator
0,0.005147,0.005147,0.861119,0.339700,0.005147,2024-05-16 08:45:00,-0.327363,-0.775276,-0.278225,135.0,0.419046,0.203736,-0.015071,-0.969386,-0.447388,-0.354527,0.161647,-0.006785,-0.815781,-0.367064,1.095749,-0.538954,-0.427848,-0.561956,-0.317400,-0.739813,345,1212,797.0,151.0,7,0.169866,-0.017586,-0.212309,-0.093355,-0.096124,-0.005861,-0.028726,-0.020727,-0.012434,...,-0.023815,-0.224904,-0.216907,1.396312,-0.014358,-0.054435,-0.040215,-0.036392,-0.073992,-0.01988,-0.008289,-0.010152,-0.285615,-0.022324,-0.005861,-0.004145,-0.656089,-0.040853,-0.016054,-0.011723,-1.551237,0.008054,-0.507494,-0.633235,0.331613,-0.680993,-0.427773,1.34184,2.69168,-0.116379,-1.381983,0.86859,-1.340544,-1.475355,0.465877,1.198827,-1.623998,-0.233263,-0.233267,0.837474
1,0.005131,0.005131,1.188921,-0.087730,0.005131,2024-05-16 08:45:00,-0.640468,0.845118,-0.567421,112.5,-0.150582,-0.093386,-0.233263,-0.028445,1.026427,-0.617642,0.161647,0.270119,1.175924,-0.331174,0.141322,-0.383361,-0.284121,-0.496389,-0.295595,-0.739813,314,1229,780.0,162.0,7,0.169866,-0.017586,-0.212309,-0.093355,-0.096124,-0.005861,-0.028726,-0.020727,-0.012434,...,-0.023815,-0.224904,-0.216907,1.396312,-0.014358,-0.054435,-0.040215,-0.036392,-0.073992,-0.01988,-0.008289,-0.010152,-0.285615,-0.022324,-0.005861,-0.004145,-0.656089,-0.040853,-0.016054,-0.011723,-1.551237,0.008054,-0.507494,-0.633235,0.331613,0.799774,-0.438197,1.34184,2.69168,-0.116379,-1.381983,0.86859,-1.340544,-0.327325,-0.327291,-0.969423,1.239755,-0.015071,-0.065428,1.314442
2,0.005131,0.005131,0.870111,0.421267,0.005131,2024-05-16 08:45:00,-0.536100,-0.775276,-1.435008,22.5,0.846267,0.426578,-0.233263,0.176107,0.289519,-0.529937,0.161647,0.270119,-1.042111,-0.323975,0.497493,-0.307463,-0.371114,-0.537086,-0.309301,-0.739813,323,1191,774.0,130.0,7,0.169866,-0.017586,-0.212309,-0.093355,-0.096124,-0.005861,-0.028726,-0.020727,-0.012434,...,-0.023815,-0.224904,-0.216907,1.396312,-0.014358,-0.054435,-0.040215,-0.036392,-0.073992,-0.01988,-0.008289,-0.010152,-0.285615,-0.022324,-0.005861,-0.004145,-0.656089,-0.040853,-0.016054,-0.011723,-1.551237,0.008054,-0.507494,-0.633235,0.331613,-0.724616,-0.424740,1.34184,2.69168,-0.116379,-1.381983,0.86859,-1.340544,-0.640424,0.090166,-0.028484,-1.419444,-0.233263,-0.233267,0.850558
3,0.005067,0.005067,0.243935,-1.511279,0.005067,2024-05-16 08:45:00,0.507582,1.534389,0.1

In [197]:
df['last_updated'] = pd.to_datetime(df['last_updated'])
df['hour'] = df['last_updated'].dt.hour
df['day'] = df['last_updated'].dt.day
df['weekday'] = df['last_updated'].dt.weekday
df['month'] = df['last_updated'].dt.month
df['year'] = df['last_updated'].dt.year

In [83]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['temperature_celsius','last_updated'])  # Features
y = df['temperature_celsius']  # Target


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (40752, 97), Test size: (17466, 97)


Baseline Model

In [85]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [86]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [87]:
y_pred = lr_model.predict(X_test)

In [88]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Baseline Model (Linear Regression):")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

Baseline Model (Linear Regression):
MAE: 0.0000
RMSE: 0.0000
R² Score: 1.0000


In [89]:
# Check if any feature is perfectly correlated with the target
correlation_check = X_train.corrwith(y_train).abs().sort_values(ascending=False)
print(correlation_check.head(10))  # Show top correlated features


feels_like_celsius       0.981143
sunrise                  0.589897
temp_diff                0.589571
distance_from_equator    0.546229
uv_index                 0.524931
pressure_mb              0.501366
latitude                 0.370432
sunset                   0.348823
month_cos                0.348047
humidity                 0.343892
dtype: float64


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [90]:
X = X.drop(columns=['feels_like_celsius'])


In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (40752, 96), Test size: (17466, 96)


In [92]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [93]:
y_pred = lr_model.predict(X_test)

In [94]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Baseline Model (Linear Regression):")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

Baseline Model (Linear Regression):
MAE: 0.3212
RMSE: 0.4200
R² Score: 0.8194


advanced modelling

In [95]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [96]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    return mae, rmse, r2

In [97]:
dt = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)

In [98]:
models = {"Decision Tree": dt, "Random Forest": rf, "XGBoost": xgb}
results = {}

In [99]:
for name, model in models.items():
    mae, rmse, r2 = evaluate_model(model, X_train, X_test, y_train, y_test)
    results[name] = {"MAE": mae, "RMSE": rmse, "R² Score": r2}

In [100]:
results_df = pd.DataFrame(results).T
print(results_df)

                    MAE      RMSE  R² Score
Decision Tree  0.129082  0.232198  0.944802
Random Forest  0.089817  0.155113  0.975368
XGBoost        0.112402  0.160066  0.973770


In [101]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [102]:
rf_model = RandomForestRegressor(random_state=42)

In [103]:
random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_grid,
    n_iter=20,  
    cv=3,   
    scoring='neg_mean_absolute_error',
    n_jobs=-1,  
    verbose=2,  
    random_state=42
)

In [104]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\base.py", line 436, in _validat

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200, 300,
                                                         500]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [105]:
#best params
print("Best Parameters:", random_search.best_params_)

# Best Model
best_rf = random_search.best_estimator_

Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}


In [106]:
y_pred = best_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Tuned Random Forest:\nMAE: {mae:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")

Tuned Random Forest:
MAE: 0.1257, RMSE: 0.1805, R² Score: 0.9667


In [107]:
import joblib

# Save model
joblib.dump(best_rf, '../models/random_forest_weather_model.pkl')

['../models/random_forest_weather_model.pkl']

Trying with ARIMA

In [198]:
from pmdarima import auto_arima

In [199]:
df_arima = df.sort_values('last_updated')

In [200]:
df_arima= df.set_index('last_updated', inplace =True)

In [201]:
df_arima = df.resample('H').mean()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15504\881722753.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_arima = df.resample('H').mean()


In [202]:
df_arima = df_arima.interpolate()

In [203]:
df_arima.head()

,country,location_name,latitude,longitude,timezone,temperature_celsius,wind_kph,wind_degree,wind_direction,pressure_mb,pressure_in,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination,condition_text_Blowing snow,condition_text_Clear,condition_text_Cloudy,condition_text_Fog,condition_text_Freezing drizzle,condition_text_Freezing fog,condition_text_Heavy rain,condition_text_Heavy rain at times,condition_text_Heavy snow,...,condition_text_Overcast,condition_text_Partly Cloudy,condition_text_Partly cloudy,condition_text_Patchy heavy snow,condition_text_Patchy light drizzle,condition_text_Patchy light rain,condition_text_Patchy light rain in area with thunder,condition_text_Patchy light rain with thunder,condition_text_Patchy light snow,condition_text_Patchy light snow in area with thunder,condition_text_Patchy moderate snow,condition_text_Patchy rain nearby,condition_text_Patchy rain possible,condition_text_Patchy snow nearby,condition_text_Patchy snow possible,condition_text_Sunny,condition_text_Thundery outbreaks in nearby,condition_text_Thundery outbreaks possible,condition_text_Torrential rain shower,hour,day,month,year,temp_diff,wind_power,air_quality_risk,hour_sin,hour_cos,day_sin,day_cos,month_sin,month_cos,temperature_celsius_lag_1,temperature_celsius_lag_7,humidity_lag_1,humidity_lag_7,precip_mm_lag_1,precip_mm_lag_7,distance_from_equator,weekday
last_updated,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-05-16 08:00:00,0.005663,0.004842,0.055110,-0.037804,0.006164,0.130032,0.000692,-0.171239,146.311475,-0.066538,-0.041431,-0.049556,-0.005419,0.013924,0.166719,0.120690,0.208080,0.099186,-0.109635,-0.161828,-0.197120,-0.118134,-0.227954,-0.139350,-0.321615,348.333333,1152.633880,791.846995,100.967213,7.0,0.169866,-0.017586,0.056676,-0.034311,0.018623,-0.005861,-0.028726,-0.020727,-0.012434,-0.029024,...,0.004829,-0.085017,0.057251,-0.014358,-0.054435,-0.040215,0.113964,0.074520,-0.01988,-0.008289,-0.010152,0.024780,-0.022324,-0.005861,-0.004145,-0.024644,0.093132,-0.016054,-0.011723,8.0,16.0,5.0,2024.0,-0.239841,0.017591,-0.193469,1.341840,2.691680,-0.116379,-1.381983,0.86859,-1.340544,0.123105,0.120506,0.007061,0.007514,-0.049556,-0.046626,-0.006792,3.0
2024-05-16 09:00:00,0.005660,0.004844,0.054593,-0.039820,0.006158,0.149745,0.055393,-0.148980,148.314387,-0.081045,-0.048613,-0.036978,-0.026913,-0.008482,0.186822,0.125751,0.222306,0.156877,-0.073854,-0.149573,-0.185793,-0.121629,-0.207669,-0.133125,-0.322416,348.398830,1152.618584,791.910215,100.984081,7.0,0.169866,-0.017586,0.050706,-0.044152,-0.000502,-0.005861,-0.028726,-0.020727,-0.012434,-0.029024,...,-0.004777,-0.090061,0.080298,-0.014358,-0.054435,-0.040215,0.113041,0.109369,-0.01988,-0.008289,-0.010152,-0.000378,-0.022324,-0.005861,-0.004145,-0.020871,0.070801,-0.016054,-0.011723,9.0,16.0,5.0,2024.0,-0.254609,0.070528,-0.164278,0.800675,2.216996,-0.116379,-1.381983,0.86859,-1.340544,0.143730,0.140570,-0.016125,-0.014526,-0.036978,-0.034492,-0.005030,3.0
2024-05-16 10:00:00,0.005657,0.004846,0.054076,-0.041836,0.006151,0.169457,0.110093,-0.126721,150.317299,-0.095552,-0.055795,-0.024399,-0.048407,-0.030888,0.206924,0.130813,0.236532,0.214568,-0.038073,-0.137319,-0.174467,-0.125123,-0.187385,-0.126901,-0.323217,348.464327,1152.603288,791.973435,101.000949,7.0,0.169866,-0.017586,0.044737,-0.053993,-0.019626,-0.005861,-0.028726,-0.020727,-0.012434,-0.029024,...,-0.014383,-0.095105,0.103345,-0.014358,-0.054435,-0.040215,0.112117,0.144217,-0.01988,-0.008289,-0.010152,-0.025537,-0.022324,-0.005861,-0.004145,-0.017098,0.048470,-0.016054,-0.011723,10.0,16.0,5.0,2024.0,-0.269376,0.123464,-0.135087,0.259511,1.742312,-0.116379,-1.381983,0.86859,-1.340544,0.164355,0.160634,-0.039310,-0.036566,-0

In [204]:
ts = df_arima['temperature_celsius'].dropna()

In [205]:
split_index = int(len(ts) * 0.8)
train_ts = ts.iloc[:split_index]
test_ts = ts.iloc[split_index:]

In [206]:
arima_model = auto_arima(
    train_ts,
    seasonal=True,
    m=24,  # 24 hourly intervals in a day
    trace=True,
    error_action='ignore',
    suppress_warnings=True
)

c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarni

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[24] intercept   : AIC=-20633.549, Time=11.88 sec


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,1,0)(0,0,0)[24] intercept   : AIC=-20648.530, Time=0.69 sec


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

 ARIMA(2,1,0)(0,0,0)[24] intercept   : AIC=-20649.431, Time=1.55 sec


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,1,2)(0,0,0)[24] intercept   : AIC=-20640.602, Time=5.81 sec


c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,1)(0,0,0)[24]             : AIC=inf, Time=3.52 sec

Best model:  ARIMA(1,1,1)(0,0,0)[24] intercept
Total fit time: 230.112 seconds


In [208]:
n_periods = len(test_ts)
forecast = arima_model.predict(n_periods=n_periods)
forecast_index = test_ts.index

c:\Users\Administrator\.conda\envs\assessmentenv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [209]:
# Replace MAPE with SMAPE
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred)
    return np.mean(diff / denominator) * 100


In [210]:
mae = mean_absolute_error(test_ts, forecast)
rmse = np.sqrt(mean_squared_error(test_ts, forecast))
smape_val = smape(test_ts.values, forecast)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"SMAPE: {smape_val:.2f}%")

MAE: 0.32
RMSE: 0.40
SMAPE: 69.93%


In [136]:
print("Min temperature in test set:", test_ts.min())
print("Some test values:", test_ts.head(10).values)

Min temperature in test set: -0.8789797727415396
Some test values: [-0.40320005 -0.40338714 -0.40357424 -0.40376133 -0.40394843 -0.40413553
 -0.40432262 -0.40450972 -0.40480201 -0.40509431]


Trying with Prophet model

In [139]:
from prophet import Prophet

In [173]:
df = pd.read_csv('../data/processed/feature_eng_weather_data.csv')
print("Processed dataset loaded successfully!")
df.columns

Processed dataset loaded successfully!


Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated', 'temperature_celsius', 'wind_kph', 'wind_degree',
       'wind_direction', 'pressure_mb', 'pressure_in', 'precip_mm', 'humidity',
       'cloud', 'feels_like_celsius', 'visibility_km', 'uv_index', 'gust_kph',
       'air_quality_Carbon_Monoxide', 'air_quality_Ozone',
       'air_quality_Nitrogen_dioxide', 'air_quality_Sulphur_dioxide',
       'air_quality_PM2.5', 'air_quality_PM10', 'air_quality_us-epa-index',
       'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase',
       'moon_illumination', 'condition_text_Blowing snow',
       'condition_text_Clear', 'condition_text_Cloudy', 'condition_text_Fog',
       'condition_text_Freezing drizzle', 'condition_text_Freezing fog',
       'condition_text_Heavy rain', 'condition_text_Heavy rain at times',
       'condition_text_Heavy snow', 'condition_text_Light drizzle',
       'condition_text_Light freezing rain', 'condition_text_Light rain',


In [174]:
df['last_updated'] = pd.to_datetime(df['last_updated'])

In [175]:
df = df.sort_values('last_updated')

In [176]:
split_index = int(len(df) * 0.8)
train_df = df.iloc[:split_index].copy()
test_df = df.iloc[split_index:].copy()

In [177]:
regressors = [
    'humidity', 'precip_mm', 'cloud',
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos',
    'temperature_celsius_lag_1', 'humidity_lag_1', 'precip_mm_lag_1'
]

In [178]:
train_df_prophet = train_df.rename(columns={'last_updated': 'ds', 'temperature_celsius': 'y'})
train_df_prophet = train_df_prophet[['ds', 'y'] + regressors]

In [179]:
model = Prophet()
for reg in regressors:
    model.add_regressor(reg)

model.fit(train_df_prophet)

15:02:17 - cmdstanpy - INFO - Chain [1] start processing
15:02:35 - cmdstanpy - INFO - Chain [1] done processing


In [180]:
test_df_prophet = test_df.rename(columns={'last_updated': 'ds'})
future = test_df_prophet[['ds'] + regressors]

In [181]:
forecast = model.predict(future)

In [182]:
y_true = test_df['temperature_celsius'].values
y_pred = forecast['yhat'].values

In [183]:
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
smape = 100 / len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

print(f"Improved Prophet → MAE: {mae:.2f} | RMSE: {rmse:.2f} | SMAPE: {smape:.2f}%")

Improved Prophet → MAE: 0.92 | RMSE: 1.06 | SMAPE: 142.61%


In [159]:
test_df['y'].loc[test_df['y'].abs() < 1].count()

7622

Trying XGBOOST

In [213]:
from xgboost import XGBRegressor

In [214]:
df = pd.read_csv('../data/processed/feature_eng_weather_data.csv')
df['last_updated'] = pd.to_datetime(df['last_updated'])

In [215]:
target = 'temperature_celsius'

In [216]:
drop_cols = ['last_updated', 'temperature_celsius', 'ds', 'y']
X = df.drop(columns=[col for col in drop_cols if col in df.columns])
y = df[target]

In [217]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # no shuffle for time-series feel
)


In [218]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [219]:
y_pred = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
smape = 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"SMAPE: {smape:.2f}%")


MAE: 0.02
RMSE: 0.05
SMAPE: 5.44%


In [212]:
import pickle
with open("../models/arima_model.pkl", "wb") as f:
    pickle.dump(arima_model, f)

In [221]:
xgb_model.save_model("../models/xgboost_model.json")